In [ ]:
epochs = 10

# Parte 6 - Aprendizaje federado en MNIST usando una red neuronal convolucional.

## Actualiza el código a aprendizaje federado en 10 líneas de Pytorch + PySyft

### Contexto 

El aprendizaje federado es una técnica del aprendizaje automático muy emocionante y en surgimiento, que busca que sistemas aprendan con datos descentralizados. La idea es que los datos permanezcan en las manos de quienes los producen (también llamados trabajadores), lo que ayuda a mejorar la privacidad y mantener la propiedad de la data y el modelo es compartido entre los trabajadores. Una aplicación inmediata es por ejemplo predecir la siguiente palabra de un celular cuando se escribe un texto: Tú no quieres que tu información sea usada para entrenamiento - por ejemplo, tus mensajes de texto - sean enviados al servidor central.

El desarrollo del aprendizaje federado está muy conectado con el aumento en la concientización de la necesidad de la privacidad en los datos y las normas de GDPR en la unión europea, en las que la UE obliga la protección de datos desde mayo de 2018. Para anticiparse a la regulación, muchos actores como apple y google han empezado a invertir masivamente en esta tecnología, especialmente para proteger la privacidad de usuarios de celulares, pero no han puesto sus herramientas disponibles.

En OpenMined, creemos que todos los que podemos realizar un proyecto de aprendizaje automático también deberíamos poder implementar herramientas para preservar la privacidad con poco esfuerzo. Nosotros hemos construido herramientas para encriptar datos en una sola línea [como es mencionado en nuestro blog post](https://blog.openmined.org/training-cnns-using-spdz/) y nosotros ahora lanzamos nuestra plataforma de aprendizaje federado que usa la nueva versión de Pytorch 1.0 para proveer con una interfaz para construir modelos seguros y escalables.

En este tutorial, usaremos directamente [el ejemplo canónico de entrenar una red neuronal convolucional en MNIST usando Pytorch](https://github.com/pytorch/examples/blob/master/mnist/main.py) y mostraremos lo simple que es implementar aprendizaje federado usando nuestra [libreria de PySyft](https://github.com/OpenMined/PySyft/). Iremos por cada parte del ejemplo y señalaremos el código que haya cambiado.

También puedes encontrar este material en [nuestro blogpost](https://blog.openmined.org/upgrade-to-federated-learning-in-10-lines).

Autores:
- Théo Ryffel - GitHub: [@LaRiffle](https://github.com/LaRiffle)

Traductores:
- Carlos Salgado - Github [@socd06](https://github.com/socd06)
- Arturo Márquez Flores - Twitter [@arturomf94](https://twitter.com/arturomf94)
- Ricardo Pretelt - Twitter [@ricardopretelt](https://twitter.com/ricardopretelt)

**Ok, ¡Empecemos!**

### Librerias a importar y especificaciones del modelo.
Primero importamos las librerías

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

Y luego lo específico a PySyft. En particular definimos los trabajadores remotos `alice` y `bob`.

In [ ]:
import syft as sy  # <-- Nuevo: importamos la librería de Pysyft
hook = sy.TorchHook(torch)  # <-- Nuevo: hook PyTorch.  agrega funciones para apoyar el aprendizaje federado
bob = sy.VirtualWorker(hook, id="bob")  # <-- Nuevo: Definir el trabajador remoto bob
alice = sy.VirtualWorker(hook, id="alice")  # <-- Nuevo: También definimos a alice

Definimos la configuración del proceso de aprendizaje.

In [ ]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = epochs
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

### Cargamos los datos y los enviamos a los trabajadores
Nosotros cargamos los datos y transformamos el conjunto de datos de entrenamiento a otro federado dividido entre trabajadores usando el método `.federate`. Estos datos federados son enviados a un _dataloader_ federado. Los datos de prueba permanecen sin cambios.


In [ ]:
federated_train_loader = sy.FederatedDataLoader( # <-- esto es ahora un DataLoader federado
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((bob, alice)), # <-- Nuevo: Distribuimos los datos en todos los trabajadores, ahora son federados.
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

### Especificaciones para la red convolucional
Aquí usamos la misma red del ejemplo oficial.

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

### Definir las funciones de entrenamiento y de prueba
Para la función de entrenamiento, como los lotes de datos están distribuidos por `alice` y `bob`, necesitas enviar el modelo al lugar correcto para cada lote. Entonces, realizas todas las operaciones remotamente con la misma sintaxis como si se estuviera realizando localmente en Pytorch. Cuando acabas, envías el modelo de vuelta actualizado y la pérdida obtenida para mejorar.

In [ ]:
def train(args, model, device, federated_train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- Ahora son datos distribuidos
        model.send(data.location) # <-- Nuevo: Enviar el modelo a la ubicación correcta
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # <-- Nuevo: Enviar el modelo de vuelta
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # <-- Nuevo: Enviar las pérdidas de vuelta
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
                100. * batch_idx / len(federated_train_loader), loss.item()))

La función de prueba no cambia!

In [ ]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sumar las pérdidas por lote
            pred = output.argmax(1, keepdim=True) # Obtener el índice de la máxima probabilidad logarítmica 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

### Lanza el entrenamiento!

In [ ]:
%%time
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) # Momentum no está disponible en el momento

for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")



Et voilà! Aquí está, has entrenado un modelo con datos remotos usando aprendizaje federado!

## Para terminar
Sé que hay una pregunta que te mueres por realizar: **¿Cuánto tiempo toma realizar aprendizaje federado comparado con PyTorch normal?**
El tiempo de computación es **menos del doble del tiempo** usado para una ejecución normal de PyTorch! Más precisamente, toma 1.9 veces más tiempo, lo que es pequeño comparado con las características que pudimos agregar.

## Conclusión

Como puedes observar, modificamos 10 líneas de código para actualizar el ejemplo oficial de Pytorch en MNIST a una configuración real de ¡aprendizaje federado!

Claro, existen una docena de mejoras que se pueden pensar. Nos gustaría que la computación se operara en paralelo de los trabajadores y realizar un promedio federado, actualizar el modelo central cada `n` lote solamente, reducir el número de mensajes usados para comunicar entre los trabajadores, etc. Estas son características en que estamos trabajando para hacer aprendizaje federado listo para producción y escribiremos sobre ellos tan pronto al estar disponibles.

Ahora deberías poder hacer aprendizaje federado por ¡tí mismo! Si disfrutaste esto, y te gustaría unirte al movimiento para preservar la privacidad, la propiedad descentralizada de AI y la cadena de suministro de AI (los datos), puedes hacerlo de ¡las siguientes formas!
 

### Dale una estrella a PySyft en GitHub

¡La forma más fácil de ayudar a nuestra comunidad es por darle estrellas a los repositorios de Github! Esto ayuda a crear consciencia de las interesantes herramientas que estamos construyendo.

- [Estrella a PySyft](https://github.com/OpenMined/PySyft)

### Usa nuestros tutoriales de GitHub!

Hicimos muy buenos tutoriales  para conseguir un mejor entendimiento de lo que aprendizaje federado y aprendizaje que preserva la privacidad debe ser y cómo construimos los bloques para que esto pase.

- [Mirar los tutoriales de PySyft](https://github.com/OpenMined/PySyft/tree/master/examples/tutorials)

### ¡Únete a nuestro Slack!

La mejor manera de mantenerte actualizado con los últimos avances es ¡unirte a la comunidad!

- [Unirse a slack.openmined.org](http://slack.openmined.org)

### ¡Únete a un proyecto de código!

La mejor manera de contribuir a nuestra comunidad es convertirte en un ¡contribuidor de código! En cualquier momento puedes ir al Github Issues de PySyft y filtrar por "Proyectos". Esto mostrará todos los tiquetes de nivel superior dando un resumen de los proyectos a los que ¡te puedes unir! Si no te quieres unir a un proyecto, pero quieres hacer un poco de código, también puedes mirar más mini-proyectos "de una persona" buscando por Github Issues con la etiqueta `Good First Issue`.

- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Donar


Si no tienes tiempo para contribuir a nuestra base de código, pero quieres ofrecer tu ayuda, también puedes aportar a nuestro Open Collective". Todas las donaciones van a nuestro web hosting y otros gastos de nuestra comunidad como ¡hackathons y meetups!

- [Donar a través de la página de OpenMined](https://opencollective.com/openmined)